# Grid Rival

In [1]:
import pandas as pd

In [2]:
from gridrival.probabilities import GridProbabilities
from gridrival.probabilities.betting_odds import WinningOdds, TOP_1
from gridrival.drivers import DRIVERS
from gridrival.fantasy import FantasyTeam
from gridrival.teams import TEAMS

In [3]:
Top1Odds = WinningOdds(pd.Series(TOP_1))

In [4]:
Prob = GridProbabilities(Top1Odds.rank_grid())

In [5]:
for driver in DRIVERS:
    driver.probabilities = Prob.driver_probabilities(driver.name)

In [6]:
TEAMS[0].driver_1.expected_points()

145.0

In [7]:
DRIVERS[0].team

Mercedes

In [8]:
ft = FantasyTeam(DRIVERS[0:5], TEAMS[0])

In [9]:
ft.cost()

167000000.0

In [10]:
ft.expected_points()

871.0